In [1]:
import simtk.openmm.app as app
import simtk.openmm as mm
import simtk.unit as unit
import mdtraj as md
import matplotlib.pyplot as plt
import nglview as nv

_ColormakerRegistry()

**Cargar el archivo PDB del disco y agregar residuos y átomos restantes**:

In [2]:
pdb =app.PDBFile("BAK_model1_all.pdb")


**Especificar el campo de fuerza a utilizar para la simulación**:

In [3]:
forcefield =app.ForceField("charmm36.xml", "charmm36/water.xml")

In [4]:
modeller = app.Modeller(pdb.topology, pdb.positions)
modeller.addSolvent(forcefield, padding=1.0*unit.nanometers)

In [5]:
app.PDBFile.writeFile(modeller.topology, modeller.positions, open("BAK_model1_all_solexp.pdb", "w"))

**Crear una descripción matematica completa del sistema por simular**:

In [6]:
system = forcefield.createSystem(modeller.topology, nonbondedMethod= app.PME,
                                 nonbondedCutoff=1*unit.nanometer, constraints=app.HBonds)

**Crear el integrador**:

Esta línea crea el integrador empleado para avanzar las ecuaciones de movimiento. 

LangevinIntegrator: realiza la dinámica de Langevin, y lo asigna a una variable llamada integrador.

Se declaran los valores de tres parámetros que son específicos de la dinámica de Langevin: la temperatura de simulación (300 K), el coeficiente de fricción (1 ps -1 ) y el tamaño del paso (2 fs).

In [7]:
integrator = mm.LangevinIntegrator(300*unit.kelvin, 1.0/unit.picoseconds, 2.0*unit.femtoseconds)
integrator.setConstraintTolerance(0.00001)

**Preparar la simulación**:

- Definir la plataforma (CUDA o CPU):

In [8]:
platform = mm.Platform.getPlatformByName('CPU')
#properties = {'CudaPrecision': 'mixed'}
properties = {}

- Crear el objeto simulación:

Esta línea combina la topología molecular, el sistema y el integrador para comenzar una nueva simulación.

In [9]:
simulation = app.Simulation(modeller.topology, system, integrator, platform, properties)

In [10]:
simulation.context.setPositions(modeller.positions)

(Esta línea especifica las posiciones iniciales de los átomos para la simulación, en este caso: las posiciones que se cargaron desde el archivo PDB.)

**Minimización de energía**:

In [11]:
Edo_PreMin = simulation.context.getState(getEnergy = True)

In [12]:
Edo_PreMin.getPotentialEnergy()

Quantity(value=-136092.19337919174, unit=kilojoule/mole)

In [13]:
print('Minimizing...')
simulation.minimizeEnergy(tolerance=0.1*unit.kilojoule/unit.mole, maxIterations=500)

Minimizing...


In [14]:
Edo_PosMin = simulation.context.getState(getEnergy = True, getPositions = True)

In [15]:
Edo_PosMin.getPotentialEnergy()

Quantity(value=-313355.9067909101, unit=kilojoule/mole)

In [16]:
positions = Edo_PosMin.getPositions()

**Guardar archivo PDB post-minimización**

In [17]:
app.PDBFile.writeFile(simulation.topology, positions, open('output_PosMin_Charmm36_ExpSol.pdb', 'w'))

**Calcular el RMSD entre las estructuras pre y post minimización**

Sólo proteína con H

In [18]:
PreMin = md.load("BAK_model1_all_solexp.pdb")
PostMin = md.load("output_PosMin_Charmm36_ExpSol.pdb")

In [19]:
lista_atomos_peptido = PreMin.topology.select("protein")

In [20]:
RMSD = md.lprmsd(PreMin, PostMin, atom_indices = lista_atomos_peptido)

In [21]:
RMSD

array([0.10014618], dtype=float32)

Sin hidrógenos

In [22]:
lista_atomos_pesados = PreMin.topology.select("protein and type != H")

In [23]:
RMSD_heavy_atomms = md.lprmsd(PreMin, PostMin, atom_indices = lista_atomos_pesados)

In [24]:
RMSD_heavy_atomms

array([0.09009279], dtype=float32)

**RMSD Backbone**

In [25]:
backbone = PreMin.topology.select("protein and backbone")

In [26]:
RMSD_backbone = md.lprmsd(PreMin, PostMin, atom_indices = backbone)

In [27]:
RMSD_backbone

array([0.07430118], dtype=float32)

**Visualizar las estructuras**